# Библиотека Rank-BM25

В этом тюториале мы научимся ранжировать документы по формуле _BM25_ с помощью библиотеки _Rank-BM25_.

Код и документация библиотеки: https://github.com/dorianbrown/rank_bm25

Ее можно поставить в виртуальное окружение командой `pip install rank-bm25`

Импортируем модули которые нам понадобятся впоследствии:

In [1]:
import numpy as np
from sklearn.metrics import pairwise
from nltk import tokenize
from rank_bm25 import BM25Okapi

Предположим, что на входе нам дана коллекция из нескольких текстовых документов (названия институтов):

In [2]:
docs = [
    "Московская государственная академия хореографии",
    "Московский государственный университет им. М.В. Ломоносова (Университет МГУ)",
    "Московский физико-технический институт (национальный исследовательский университет)",
    "Национальный исследовательский университет «МИЭТ»",
    "Национальный исследовательский университет ИТМО",
]
docs

['Московская государственная академия хореографии',
 'Московский государственный университет им. М.В. Ломоносова (Университет МГУ)',
 'Московский физико-технический институт (национальный исследовательский университет)',
 'Национальный исследовательский университет «МИЭТ»',
 'Национальный исследовательский университет ИТМО']

## Предобработка текста

Библиотека _rank_bm25_, в отличие от векторизаторов из библиотеки _scikit-learn_, не умеет сама токенизировать текст, поэтому нам придется сделать это самостоятельно.

Для этого будем использовать токенизатор из библиотеки _NLTK_.<br>
Напишем вспомогательную функцию, которая сначала токенизирует текст, а потом приводит каждый токен к нижнему регистру:

In [3]:
def preprocess(text):
    # Tokenize
    tokenizer = tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    # Normalize
    return [token.lower() for token in tokens]

И применим ее к каждому из наших текстовых документов:

In [4]:
prep_docs = [preprocess(doc) for doc in docs]
print(prep_docs)

[['московская', 'государственная', 'академия', 'хореографии'], ['московский', 'государственный', 'университет', 'им', 'м', 'в', 'ломоносова', 'университет', 'мгу'], ['московский', 'физико', 'технический', 'институт', 'национальный', 'исследовательский', 'университет'], ['национальный', 'исследовательский', 'университет', 'миэт'], ['национальный', 'исследовательский', 'университет', 'итмо']]


Мы видим, что каждый документ теперь представлен в виде списка из терминов.

## Ранжирование с помощью BM25

Теперь перейдем к самому интересному: попробуем ранжировать наши документы, используя в качестве ранков скор, который выдает формула BM25.

Сначала нам придется создать объект класса _BM25Okapi_, который "под капотом" посчитает по корпусу документов IDFы и подготовит все необходимое для применения _BM25_ к парам запрос-документ:

In [5]:
bm25 = BM25Okapi(prep_docs)
print(vars(bm25))

{'k1': 1.5, 'b': 0.75, 'epsilon': 0.25, 'corpus_size': 5, 'avgdl': 5.6, 'doc_freqs': [{'московская': 1, 'государственная': 1, 'академия': 1, 'хореографии': 1}, {'московский': 1, 'государственный': 1, 'университет': 2, 'им': 1, 'м': 1, 'в': 1, 'ломоносова': 1, 'мгу': 1}, {'московский': 1, 'физико': 1, 'технический': 1, 'институт': 1, 'национальный': 1, 'исследовательский': 1, 'университет': 1}, {'национальный': 1, 'исследовательский': 1, 'университет': 1, 'миэт': 1}, {'национальный': 1, 'исследовательский': 1, 'университет': 1, 'итмо': 1}], 'idf': {'московская': 1.0986122886681098, 'государственная': 1.0986122886681098, 'академия': 1.0986122886681098, 'хореографии': 1.0986122886681098, 'московский': 0.33647223662121295, 'государственный': 1.0986122886681098, 'университет': 0.19794868164121474, 'им': 1.0986122886681098, 'м': 1.0986122886681098, 'в': 1.0986122886681098, 'ломоносова': 1.0986122886681098, 'мгу': 1.0986122886681098, 'физико': 1.0986122886681098, 'технический': 1.098612288668

Видим, что в объекте _BM25Okapi_ лежат:
- гиперпараметры формулы _BM25_ такие как _k1_ и _b_
- частоты (_doc_freqs_) и IDFы (_idf_) терминов
- другая полезная информация, например средняя длина документа _avgdl_, которая используется в формуле _BM25_

Применим наш объект к запросу "университет".<br>
Обратите внимание, что запросы теперь тоже надо предобрабатывать с помощью функции _preprocess()_.

In [8]:
# Представим запрос в виде списка терминов
prep_query = preprocess("университет")

# Получим для каждого документа ранки, посчитанные по формуле BM25
scores = bm25.get_scores(prep_query)
scores

array([0.        , 0.23660888, 0.1779314 , 0.22715422, 0.22715422])

Видим, что самый большой ранк у 1-го документа (а это МГУ, как и ожидалось).

Теперь легко отсортировать документы по ранку, но у объекта _BM25Okapi_ уже есть удобная функция, которая сделает все за нас:

In [9]:
search_results = bm25.get_top_n(prep_query, docs, n=5)
search_results

['Московский государственный университет им. М.В. Ломоносова (Университет МГУ)',
 'Национальный исследовательский университет ИТМО',
 'Национальный исследовательский университет «МИЭТ»',
 'Московский физико-технический институт (национальный исследовательский университет)',
 'Московская государственная академия хореографии']

Теперь попробуем подать многословный запрос:

In [10]:
prep_query = preprocess("московский университет")
scores = bm25.get_scores(prep_query)
print(scores)
search_results = bm25.get_top_n(prep_query, docs, n=5)
print(search_results)

[0.         0.5008788  0.48037835 0.22715422 0.22715422]
['Московский государственный университет им. М.В. Ломоносова (Университет МГУ)', 'Московский физико-технический институт (национальный исследовательский университет)', 'Национальный исследовательский университет ИТМО', 'Национальный исследовательский университет «МИЭТ»', 'Московская государственная академия хореографии']


Таким образом, мы научились ранжировать документы с использованием формулы _BM25_!